<a href="https://colab.research.google.com/github/rohit-447/Py-Torch-Learning/blob/main/CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Loding Modules
import torch
from torch import nn
import torchvision
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
torch.__version__

In [ ]:
#@title DataLoader

#Downloading and transforming data into tensor
#Train Data
train_data=FashionMNIST(
    root='data',
    download=True,
    transform=ToTensor(),
    train=True
)

#Test Data
test_data=FashionMNIST(
    root='data',
    download=True,
    transform=ToTensor(),
    train=False
)

print(f"\nTrain Data:{len(train_data)}\nTest Data:{len(test_data)}")


In [ ]:
#@title Visulisation of Data

#getting the class of the data
class_idx=FashionMNIST.classes

#displaying a single iamge
img,label=train_data[0]
plt.imshow(img.squeeze(), cmap="grey")
plt.title(class_idx[label])
plt.axis(False)

#displying the multiple images
random_idx=torch.randint(0,len(train_data),size=[1])
plt.figure(figsize=(8,8))
rows=2
cols=5
for i in range(1,rows*cols+1):
  img,label=train_data[i]
  plt.subplot(rows,cols,i)
  plt.imshow(img.squeeze(), cmap="grey")
  plt.title(class_idx[label])
  plt.axis(False)

print(f"Shape of the image:{img.shape}")

In [ ]:
#@title Device Agonistic Code
device="cuda" if torch.cuda.is_available() else "cpu"
print(f'You are on device: {device}')

In [ ]:
#@title DataLoader

#from torch.utils.data import DataLoader
BATCH_SIZE=32
train_dataloader=DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader=DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
#@title Visulaisation of data in dataloader
plt.figure(figsize=(10,10))
rows=8
cols=4
# Get a batch of images and labels
images, labels = next(iter(train_dataloader))

for i in range(rows*cols):
  plt.subplot(rows, cols, i+1)
  plt.imshow(images[i].squeeze(), cmap="grey")
  plt.title(class_idx[labels[i]])
  plt.axis(False)

plt.tight_layout()
plt.show()

In [ ]:
#@title Model using CNN
class MNISTModel(nn.Module):
  def __init__(self,
               input_shape:int,
               hidden_units:int,
               ouput_shape:int):

    super().__init__()
    #layer1
    self.layer_stack_1=nn.Sequential(

        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=2,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=2,
                  padding=1),
        nn.MaxPool2d(kernel_size=2, stride=2))

    #Layer 2
    self.layer_stack_2=nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=2,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2))

    #layer 3
    self.classifier=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*1*1,
                  out_features=ouput_shape))

  def forward(self, x: torch.Tensor):
        x = self.layer_stack_1(x)
        x = self.layer_stack_2(x)
        x = self.classifier(x)
        return x

model=MNISTModel(input_shape=1,
                 hidden_units=10,
                 ouput_shape=len(class_idx)).to(device)

In [ ]:
#@title Optimizer Loss fn and Accuracy Score
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(params=model.parameters(), lr=0.1)
def acc_fn(y_true, y_pred):
  correct=torch.eq(y_true, y_pred).sum().item()
  acc=(correct/len(y_pred))*100
  return acc

In [ ]:
#@title Training and Testing Loop
from tqdm.auto import tqdm
EPOCH=3
for epoch in tqdm(range(EPOCH)):
  train_loss=0
  train_acc=0
  test_loss=0
  model.train()

  for batch, (X,y) in enumerate(train_dataloader):
    X,y = X.to(device), y.to(device)


    #forward Pass
    y_pred=model(X)

    #loss and acc
    loss=loss_fn(y_pred,y)
    train_loss+=loss.item()

    preds = y_pred.argmax(dim=1)
    train_acc += acc_fn(y_true=y, y_pred=preds)

    #optimize the zero grad
    optimizer.zero_grad()

    #loss backward
    loss.backward()

    #step optimizer
    optimizer.step()

    if batch %400==0:
      print(f"Looked at Epoch:[{epoch+1}] {batch*len(X)}/ {len(train_dataloader.dataset)} Samples")

  train_loss/=len(train_dataloader)
  train_acc/=len(train_dataloader)
  model.eval()
  test_acc=0
  test_loss=0

  with torch.inference_mode():
    for X,y in test_dataloader:
      X, y = X.to(device), y.to(device)
      test_pred=model(X)
      test_loss+=loss_fn(test_pred, y).item()
      test_acc+=acc_fn(y_true=y, y_pred=test_pred.argmax(dim=1))
    test_loss/=len(test_dataloader)
    test_acc/=len(test_dataloader)
  print(f"Train loss: {train_loss:.5f} | Train acc: {train_acc:.2f}% | Test loss: {test_loss:.5f} | Test Acc: {test_acc:.2f}%\n")

In [ ]:
# Import tqdm for progress bar
from tqdm.auto import tqdm

# 1. Make predictions with trained model
y_preds = []
model.eval()
with torch.inference_mode():
  for X, y in tqdm(test_dataloader, desc="Making predictions"):
    # Send data and targets to target device
    X, y = X.to(device), y.to(device)
    # Do the forward pass
    y_logit = model(X)
    # Turn predictions from logits -> prediction probabilities -> predictions labels
    y_pred = torch.softmax(y_logit, dim=1).argmax(dim=1) # note: perform softmax on the "logits" dimension, not "batch" dimension (in this case we have a batch size of 32, so can perform on dim=1)
    # Put predictions on CPU for evaluation
    y_preds.append(y_pred.cpu())
# Concatenate list of predictions into a tensor
y_pred_tensor = torch.cat(y_preds)

In [ ]:
#@title Model Evuolation
#!pip install -q torchmetrics -U mlxtend

import mlxtend
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

confmatrix= ConfusionMatrix(num_classes=len(class_idx), task='multiclass')
confmarix_tensor=confmatrix(preds=y_pred_tensor, target=test_data.targets)

fig, ax = plot_confusion_matrix(
    conf_mat=confmatrix.numpy(),
    class_names=class_idx,
    figsize=(10, 7)
);
